In [1]:
!pip install scipy

In [3]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [26]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical = pd.read_csv('clean_fifaworldcup_historical_data.csv')


In [28]:
df_historical

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
923,England,France,2022,1,2,3
924,Argentina,Croatia,2022,3,0,3
925,France,Morocco,2022,2,0,2
926,Croatia,Morocco,2022,2,1,3


In [30]:
# Calculate Team Strength
# split df into df_home and df_away

df_home = df_historical[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical[['AwayTeam','HomeGoals','AwayGoals']]

In [36]:
# rename columns

df_home = df_home.rename(columns={'HomeTeam':'Team','HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team','HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'})

In [56]:
# concat df_home and df_away, group by teams and calculate Mean

df_Team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()

In [58]:
df_Team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.752941,1.164706
Australia,0.850000,1.850000
Austria,1.576923,1.692308
...,...,...
Uruguay,1.553571,1.303571
Wales,0.625000,1.250000
West Germany,2.050847,1.254237


In [90]:
def predict_point(Home,Away):
    if Home in df_Team_strength.index and Away in df_Team_strength.index:
        #GoalsScored * GoalsConceded
        lamb_home = df_Team_strength.at[Home,'GoalsScored']*df_Team_strength.at[Away,'GoalsConceded']
        lamb_away = df_Team_strength.at[Away,'GoalsScored']*df_Team_strength.at[Home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals away team
            for y in range(0,11):
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p

        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [94]:
# Testing Function
predict_point('Austria','Germany')
predict_point('England','Australia')

(2.4465176997538935, 0.4093189560348695)

,GoalsScored,GoalsConceded
Team,,
Algeria,13,19
Angola,1,2
Argentina,149,99
Australia,17,37
Austria,41,44
...,...,...
Uruguay,87,73
Wales,5,10
West Germany,121,74


In [76]:
Total_goals= pd.concat([df_home, df_away], ignore_index=True).groupby('Team').sum
Total_goals()


,GoalsScored,GoalsConceded
Team,,
Algeria,13,19
Angola,1,2
Argentina,149,99
Australia,17,37
Austria,41,44
...,...,...
Uruguay,87,73
Wales,5,10
West Germany,121,74
